In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from fuzzywuzzy import fuzz

//anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
outfits = pd.read_csv('outfit_combinations.csv')

In [3]:
# Outfit compiler for fuzzywuzzy
def outfit_compiler_fuzzy(text):
    # pull product_id from input
    product_id = re.search(r'\(([^)]+)', text).group(1)
    # get fuzzywuzzy score between input and all product ids
    outfits['fuzzy'] = outfits['product_id'].apply(lambda x:fuzz.partial_ratio(x, text))
    # find product_id with max fuzzywuzzy score
    found_product = outfits[outfits['fuzzy'] == outfits['fuzzy'].max()].iloc[0,1]
    # found_product_desc takes found_product and puts it  in the same format as the input
    found_product_desc = outfits[outfits['product_id'] == found_product].iloc[0,2] + ': ' + outfits[outfits['product_id'] == found_product].iloc[0,3] + ' ' + outfits[outfits['product_id'] == found_product].iloc[0,4] + ' (' + outfits[outfits['product_id'] == found_product].iloc[0,1] + ')'
    if product_id != found_product: # if the input isn't in the outfits df
        print('Product ID not found--did you mean', found_product_desc, '?\n')
    outfit_list = outfits['outfit_id'][outfits['product_id'] == found_product] # get list of all outfits where product id shows up
    outfit_num = 1 #to print the outfit number
    # iterate through each outfit
    for outfit in outfit_list:
        outfit_size = outfits[outfits['outfit_id'] == outfit].shape[0] #get how many products in the outfit
        print('Outfit Recommendation', outfit_num)
        outfit_num +=1
        # print each piece in the outfit
        print(found_product_desc) #print the input as the first product in the outfit
        for i in range(outfit_size): #print the rest of the products in the outfit
            if (outfits[outfits['outfit_id'] == outfit].iloc[i,1] != found_product):
                piece = outfits[outfits['outfit_id'] == outfit].iloc[i,2] + ': ' + outfits[outfits['outfit_id'] == outfit].iloc[i,3] + ' ' + outfits[outfits['outfit_id'] == outfit].iloc[i,4] + ' (' + outfits[outfits['outfit_id'] == outfit].iloc[i,1] + ')'
                print(piece)
        print('\n')

In [4]:
# Tfidf outfit compiler with fuzzywuzzy
def outfit_compiler_final(text):
    text = text
    tokens = outfits['product_id']
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(tokens)
    terms = vectorizer.get_feature_names()
    # pull product_id from text
    product_id = re.search(r'\(([^)]+)', text).group(1)
    Q = vectorizer.transform([product_id]) # vectorize the query
    results=cosine_similarity(X,Q).reshape((-1,1))
    if(results.sum() >0):
    #Where ever cosine similarity is a direct match we tried to find index of that row
        boolArr = (results == 1)
        index = np.where(boolArr)[0]
    # get list of outfits associated with product_id
        found_product = outfits.iloc[index,1].reset_index(drop=True)[0]
    #get outfit that involve this product
        found_product_outfit = outfits.iloc[index,0]
        #print(found_product_outfit) testing output
    # found_product_desc takes found_product and puts it  in the same format as the input
        found_product_desc = outfits[outfits['product_id'] == found_product].iloc[0,2] + ': ' + outfits[outfits['product_id'] == found_product].iloc[0,3] + ' ' + outfits[outfits['product_id'] == found_product].iloc[0,4] + ' (' + outfits[outfits['product_id'] == found_product].iloc[0,1] + ')'
        #print(found_product_desc) testing output
        outfit_list = outfits['outfit_id'][outfits['product_id'] == found_product] # get list of all outfits where product id shows up
        outfit_num = 1 #to print the outfit number
    # iterate through each outfit
        for outfit in outfit_list:
            outfit_size = outfits[outfits['outfit_id'] == outfit].shape[0] #get how many products in the outfit
            print('Outfit Recommendation', outfit_num)
            outfit_num +=1
        # print each piece in the outfit
            print(found_product_desc) #print the input as the first product in the outfit
            for i in range(outfit_size): #print the rest of the products in the outfit
                if (outfits[outfits['outfit_id'] == outfit].iloc[i,1] != found_product):
                    piece = outfits[outfits['outfit_id'] == outfit].iloc[i,2] + ': ' + outfits[outfits['outfit_id'] == outfit].iloc[i,3] + ' ' + outfits[outfits['outfit_id'] == outfit].iloc[i,4] + ' (' + outfits[outfits['outfit_id'] == outfit].iloc[i,1] + ')'
                    print(piece)
            print('\n')
    else:
        outfit_compiler_fuzzy(text)
    #print('Product ID not found--did you mean', found_product_desc, '?')

In [5]:
test_wrong = 'something random that maybe I would want (23049nnvlsjerou293094kljd)'
test1 = 'bottom: Eileen Fisher Slim Knit Skirt (01DMBRYVA2P5H24WK0HTK4R0A1)'
test2 = 'shoe: Tory Burch Penelope Mid Cap Toe Pump (01DMBRYVA2ZFDYRYY5TRQZJTBD)'

In [6]:
outfit_compiler_final(test_wrong)

Product ID not found--did you mean accessory2: Loewe Woven Fringe Open-Knit Wool Cardigan (01DVRYN4B93DG09MF4YS8P5PSJ) ?

Outfit Recommendation 1
accessory2: Loewe Woven Fringe Open-Knit Wool Cardigan (01DVRYN4B93DG09MF4YS8P5PSJ)
accessory1: BY FAR Jamie Mini Duchesse-Satin and Croc-Effect Leather Shoulder Bag (01DT2DA9XY52F7XQH552ZJW43K)
onepiece: Ulla Johnson Rosalie Tiered Cotton-Silk Blend Dress (01DVRYJHEHWV94XKY2F9HFE601)
shoe: Prada Mary Jane Mules (01DVS0C77R86TSCW59G1KNXP72)


Outfit Recommendation 2
accessory2: Loewe Woven Fringe Open-Knit Wool Cardigan (01DVRYN4B93DG09MF4YS8P5PSJ)
accessory1: BY FAR Jamie Mini Duchesse-Satin and Croc-Effect Leather Shoulder Bag (01DT2DA9XY52F7XQH552ZJW43K)
shoe: Jimmy Choo Romy 100 suede pumps (01DVA80YRD3A05GQDNZ93TVVVX)
onepiece: Ulla Johnson Rosalie Tiered Cotton-Silk Blend Dress (01DVRYJHEHWV94XKY2F9HFE601)


Outfit Recommendation 3
accessory2: Loewe Woven Fringe Open-Knit Wool Cardigan (01DVRYN4B93DG09MF4YS8P5PSJ)
accessory1: BY FAR Jam

In [7]:
outfit_compiler_final(test1)

Outfit Recommendation 1
bottom: Eileen Fisher Slim Knit Skirt (01DMBRYVA2P5H24WK0HTK4R0A1)
top: Eileen Fisher Rib Mock Neck Tank (01DMBRYVA2PEPWFTT7RMP5AA1T)
accessory1: kate spade new york medium margaux leather satchel (01DMBRYVA2S5T9W793F4CY41HE)
shoe: Tory Burch Penelope Mid Cap Toe Pump (01DMBRYVA2ZFDYRYY5TRQZJTBD)


Outfit Recommendation 2
bottom: Eileen Fisher Slim Knit Skirt (01DMBRYVA2P5H24WK0HTK4R0A1)
top: Eileen Fisher Rib Mock Neck Tank (01DMBRYVA2PEPWFTT7RMP5AA1T)
shoe: Tory Burch Penelope Mid Cap Toe Pump (01DMBRYVA2ZFDYRYY5TRQZJTBD)
accessory1: Nina Crystal Clutch (01DMHCNT41E14QWP503V7CT9G6)




In [8]:
outfit_compiler_final(test2)

Outfit Recommendation 1
shoe: Tory Burch Penelope Mid Cap Toe Pump (01DMBRYVA2ZFDYRYY5TRQZJTBD)
bottom: Eileen Fisher Slim Knit Skirt (01DMBRYVA2P5H24WK0HTK4R0A1)
top: Eileen Fisher Rib Mock Neck Tank (01DMBRYVA2PEPWFTT7RMP5AA1T)
accessory1: kate spade new york medium margaux leather satchel (01DMBRYVA2S5T9W793F4CY41HE)


Outfit Recommendation 2
shoe: Tory Burch Penelope Mid Cap Toe Pump (01DMBRYVA2ZFDYRYY5TRQZJTBD)
bottom: Eileen Fisher Slim Knit Skirt (01DMBRYVA2P5H24WK0HTK4R0A1)
top: Eileen Fisher Rib Mock Neck Tank (01DMBRYVA2PEPWFTT7RMP5AA1T)
accessory1: Nina Crystal Clutch (01DMHCNT41E14QWP503V7CT9G6)


Outfit Recommendation 3
shoe: Tory Burch Penelope Mid Cap Toe Pump (01DMBRYVA2ZFDYRYY5TRQZJTBD)
onepiece: Equipment Chemelle Midi Dress (01DMBRYVA2Q2ST7MNYR6EEY4TK)
accessory1: Nina Crystal Clutch (01DMHCNT41E14QWP503V7CT9G6)


Outfit Recommendation 4
shoe: Tory Burch Penelope Mid Cap Toe Pump (01DMBRYVA2ZFDYRYY5TRQZJTBD)
onepiece: Equipment Chemelle Midi Dress (01DMBRYVA2Q2ST7MNY